In [1]:
#Write a class (e.g. named UserItemData) in which you will store the read data. 
#I suggest that you have a path argument and three optional arguments in the class constructor: 
 #   from_date (from which date to read data), to_date (by which date to read data), 
  #  and min_ratings (minimum number of ratings a movie should have). Add a method to the 
   # class that tells you how many ratings it has read. I suggest that you add a method that saves the 
    #read data or read the data using the pickle library
import pandas as pd

class UserItemData():
    def __init__(self, *args):
        # if args is 1 - path
        if len(args) == 1:
            self.df = pd.read_csv(args[0], sep='\t')
        else:
            self.path = args[0]
            self.from_date = args[1]
            self.to_date = args[2]
            self.min_ratings = args[3]
            self.fd_list = self.from_date.split(".")
            self.td_list = self.to_date.split(".")
            #self.inserted_fd = datetime.datetime(int(self.fd_list[2]), int(self.fd_list[1]), int(self.fd_list[0]))
            
            self.df1 = pd.read_csv(self.path, sep='\t')
            #dates applied BAD
            self.cond = self.df1.loc[(self.df1['date_year'] >= int(self.fd_list[2])) & (self.df1['date_year'] <= int(self.td_list[2]))]
            self.df2 = self.cond.groupby('movieID')["rating"].count().reset_index(name="count")
            self.movies_enough_ratings = self.df2["movieID"][self.df2["count"] > self.min_ratings]
            self.movies_list = self.movies_enough_ratings.to_list()
            self.df = self.cond[self.cond['movieID'].isin(self.movies_list)]
    def nratings(self):
        return len(self.df.index)

uim1 = UserItemData('data/user_ratedmovies.dat', "12.10.2003", "12.10.2005", 120)
uim = UserItemData('data/user_ratedmovies.dat')
#print(uim1.df2)
#print(lolo1.df.head(5))
#print(lolo1.cond)
#print(lolo1.td_list[2])
#print(uim1.nratings())    
#print(uim1.df)

In [2]:
#Write a class that reads the movie file and has a get_title(movieID) method that
#returns its title for a given movie ID. Example:

#md = MovieData('data/movies.dat')
#print(md.get_title(1))
#Output:
#Toy story
import pandas as pd
class MovieData():
    def __init__(self, path):
        self.path = path
        self.df = pd.read_csv(self.path, sep='\t')
    def get_title(self, num):
        return ((self.df['title'])[self.df.id == num].to_string(index=False))
        #return self.df['title'][self.df.id == num]
    
md1 = MovieData("data/movies.dat")
print(md1.get_title(65011))

Zona Zamfirova


In [3]:
#The word "predictor" will be used to denote classes that, in a certain way, assess for a certain user 
#the rating he would give to the movies or the products available to him. 
#These classes will have a fit(self, X) method, where X is of type UserItemData, 
#and a predict(self, user_id) method, where user_id is the user ID. 
#We will use the fit method to learn the model, and the predict to calculate the 
#recommended values for a given user. Recommended values can be e.g. returned as a dictionary.

#Random predictor (+)
#Write a RandomPredictor class that accepts a minimum and maximum rating in the constructor, 
#and the predict method returns a random value (between min and max) for each product.

#md = MovieData('data/movies.dat')
#uim = UserItemData('data/user_ratedmovies.dat') 
#rp = RandomPredictor(1, 5) rp.fit(uim) 
#pred = rp.predict(78) 
#print(type(pred)) 
#items = [1, 3, 20, 50, 100] 
#for item in items: 
#    print("Movie: {}, score: {}".format(md.get_title(item), pred[item]))
import random
#1 : 10198
class RandomPredictor():
    def __init__(self, min_num, max_num):
        self.min_num = min_num
        self.max_num = max_num
    def fit(self, frame):
        self.df = frame.df
    def predict(self, user_id):
        self.user_rates = self.df[self.df['userID'] == user_id]
        self.movie = self.user_rates['movieID'].to_list()
        #otherwise insert movies table to get number of movies
        #all movies
        self.full_list = self.df['movieID'].to_list()
        self.to_watch = self.get_unwatched_movies(self.full_list, self.movie)
        
        #fill random rates to movies that "user_id" havent watched yet
        dicti = {}
        for i in self.to_watch:
                dicti[i] = random.randint(self.min_num,self.max_num)
        return dicti
    def predict_seen(self, user_id):
        self.user_rates = self.df[self.df['userID'] == user_id]
        self.movie = self.user_rates['movieID'].to_list()
        dicti = {}
        for i in self.movie:
                dicti[i] = random.randint(self.min_num,self.max_num)
        return dicti
    #enter all movies so we can find out which user havent seen yet
    def movies_counter(self, movies_frame):
        self.num_movies = len(self.df.index)
    def get_unwatched_movies(self, all_movies, seen):
        new_movies = all_movies
        for s in seen:
            if s in all_movies:
                new_movies.remove(s)
        return new_movies


md = MovieData('data/movies.dat')
uim = UserItemData('data/user_ratedmovies.dat') 
rp = RandomPredictor(1, 5) 
rp.fit(uim) 
pred = rp.predict(78) 
print(type(pred)) 
items = [1, 3, 20, 50, 100]
#prints predictions for movies with some scores
for item in items: 
    print("Movie: {}, score: {}".format(md.get_title(item), pred[item]))
    


<class 'dict'>
Movie: Toy story, score: 5
Movie: Grumpy Old Men, score: 3
Movie: Money Train, score: 1
Movie: The Usual Suspects, score: 2
Movie: City Hall, score: 2


In [4]:
#Recommendation (+)
#Write a class Recommender. The class should accept the predictor in the constructor and have two methods: 
#fit(self, X) and recommend(self, userID, n = 10, rec_seen = True). 
#The first method has the same function as in the predictor, while the second method returns an edited list of 
#recommended products for the userID user. The parameter n determines the number of recommended movies, 
#and with rec_seen we determine whether we want the already watched movies (those to which the user has 
#already given a rating) or not.
import random

class Recommender():
    def __init__(self, predictor):
        self.predictor = predictor
    def fit(self, x):
        self.rates_frame = x.df
    def recommend(self, userID ,n ,rec_seen):
        if (rec_seen):
            #movies user has seen as dict
            self.seen_movies_dict = self.predictor.predict_seen(userID)
            len_dict = len(self.seen_movies_dict) 
            remove = len_dict - n 
            #limit to n movies
            cnt = 0
            for i in range(1, remove):
                self.seen_movies_dict.pop(list(self.seen_movies_dict)[random.randint(1, (len - 1) - cnt)])
                cnt = cnt + 1
            return self.seen_movies_dict
        else:
            #movies not seen yet - > dict
            self.not_seen = self.predictor.predict(userID)
            len_dict = len(self.not_seen)
            remove = len_dict - n 
            #limit to n movies
            cnt = 0
            for i in range(1, remove):
                self.not_seen.pop(list(self.not_seen)[random.randint(1, (len_dict - 1) - cnt)])
                cnt = cnt + 1
            return self.not_seen
        
mdd = MovieData('data/movies.dat') 
uim_r = UserItemData('data/user_ratedmovies.dat') 
rp_r = RandomPredictor(1, 5)
rp_r.fit(uim_r) 
rec = Recommender(rp_r) 
rec.fit(uim_r) 
rec_items = rec.recommend(78, n=5, rec_seen=False) 
for idmovie, val in rec_items.items(): 
    print("Movie: {}, score: {}".format(md.get_title(idmovie), val)) 

Movie: Grumpy Old Men, score: 4
Movie: Ae Fond Kiss..., score: 3
Movie: The Navigator: A Mediaeval Odyssey, score: 1
Movie: Tex, score: 4
Movie: Night Patrol, score: 1
Movie: H�xan, score: 2


Average predictor (+)
Write the class AveragePredictor, which takes the parameter b in the constructor, where b >= 0. In the fit method, calculate the average for each movie according to the formula avg = (vs + b * g_avg) / (n + b), where:

vs is the sum of all ratings for this movie,  
n is the number of scores this movie received,  
g_avg is the average of all movies,  
b is the parameter of the formula for the average. If b=0, it is a simple average  
If we use AveragePredictor(b=0) in the recommender and the rest is the same as in the previous section, we get:  

Movie: Sonnenallee, score: 5.0  
Movie: Vals Im Bashir, score: 5.0  
Movie: Britannia Hospital, score: 5.0  
Movie: Il mio viaggio in Italia, score: 5.0  
Movie: Shu dan long wei, score: 5.0 
However, if we use AveragePredictor(b=100), the values change: 

Movie: The Usual Suspects, score: 4.225944245560473  
Movie: The Godfather: Part II, score: 4.146907937910189  
Movie: Cidade de Deus, score: 4.116538340205236  
Movie: The Dark Knight, score: 4.10413904093503  
Movie: 12 Angry Men, score: 4.103639627096175 

In [5]:
class AveragePredictor():
    def __init__(self, b):
        self.b = b
    def fit(self, frame):
        #UserItemData frame that we insert
        self.movies_object = MovieData("data/movies.dat")
        #movie id's
        self.movies = self.movies_object.df['id'].to_list()
        #original data frame with rates 
        self.df = frame.df
        #sum of all ratings for each movie
        self.sums = self.df.groupby('movieID')["rating"].sum().reset_index(name="sum")
        #cnt of ratings per each movie
        self.counts = self.df.groupby('movieID')["rating"].count().reset_index(name="count")
        #avg rate
        self.all_average = self.df["rating"].mean()
        dicti = {}
        for i in self.movies:
            #(vs + b * g_avg) / (n + b),
            summ = self.sums['sum'][self.sums["movieID"] == i ]
            cnt = self.counts['count'][self.counts["movieID"] == i ]
            #insert in dict  ->> movieID : average
            dicti[i] = (summ + self.b * self.all_average) / (cnt * self.b)
        return dicti



In [21]:
uim_avg = UserItemData('data/user_ratedmovies.dat')
ap = AveragePredictor(0)
avg_calc = ap.fit(uim_avg) 
items = [4, 67, 34, 585, 3453]
#prints predictions for movies with some scores
for item in items: 
    print("Movie: {}, score: {}".format(md.get_title(item), avg_calc[item].to_string(index=False)))
    
    

Movie: Waiting to Exhale, score: inf
Movie: Two Bits, score: inf
Movie: Babe, score: inf
Movie: The Brady Bunch Movie, score: inf
Movie: Here on Earth, score: inf


Recommending the most watched movies (+)
Write a ViewsPredictor that returns the number of views for each movie. This is the recommendation of most viewed movies. For our example, you will get the following:

rtAudienceNumRatings

Movie: The Lord of the Rings: The Fellowship of the Ring, score: 1576 

Movie: The Lord of the Rings: The Two Towers, score: 1528

Movie: The Lord of the Rings: The Return of the King, score: 1457 

Movie: The Silence of the Lambs, score: 1431 

Movie: Shrek, score: 1404

In [19]:
class ViewsPredictor():
    def fit(self, frame):
        self.df = frame.df
    def predict(self):
        self.movies = self.df['id'].to_list()
        dicti = {}
        for i in self.movies:
            views = self.df['rtAudienceNumRatings'][self.df['id'] == i]
            dicti[i] = views
        return dicti
            
vp = ViewsPredictor()
#movies data created few cells above - mdd
vp.fit(mdd)
dictionary = vp.predict()
items = [4, 67, 34, 585, 3453]
#prints predictions for movies with some scores
for item in items: 
    print("Movie: {}, score: {}".format(md.get_title(item), dictionary[item].to_string(index=False)))